<a href="https://colab.research.google.com/github/Yuki3153/diveintocode-ml/blob/master/word_macthing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite

In [2]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [ ]:
import MeCab
import subprocess
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import codecs
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openpyxl
from openpyxl import Workbook

cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))
tagger = MeCab.Tagger("-Owakati")
%cd "drive/My Drive/"

In [ ]:
# function

def similar(txt1, txt2):
    """seq dif ratio"""
    return SequenceMatcher(None, txt1, txt2).ratio()


def replace_(text):
    """text normalization"""
    text = re.sub("\(|\)", "", text)
    if '株式会社' in text:
        text = text.replace('株式会社',"")
    elif '有限会社' in text:
        text = text.replace('有限会社',"")
    elif '一般社団法人' in text:
        text = text.replace('一般社団法人',"")
    elif '医療法人' in text:
        text = text.replace('医療法人',"")
    elif '社会福祉法人' in text:
        text = text.replace('社会福祉法人',"")
    elif '公益財団法人' in text:
        text = text.replace('公益財団法人',"")
    elif '社会医療法人' in text:
        text = text.replace('社会医療法人',"")
    elif '地方独立行政法人' in text:
        text = text.replace('地方独立行政法人',"")
    elif '国立大学法人' in text:
        text = text.replace('国立大学法人',"")
    elif '国立療養所' in text:
        text = text.replace('国立療養所',"")
    elif '特別養護老人ホーム' in text:
        text = text.replace('特別養護老人ホーム',"")
    return text


def tf_idf(corpus_tag,target_lst):
    """tf-idf"""
    vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
    transformer = TfidfTransformer()
    tf = vectorizer.fit_transform(corpus_tag)
    tfidf = transformer.fit_transform(tf)
    tfid_output_lst = list()

    for target in tqdm(target_lst):
    target_tf = vectorizer.transform([tagger.parse(target).strip()])
    target_tfidf = transformer.transform(target_tf)
    similarity = cosine_similarity(target_tfidf, tfidf)[0]
    topn_indices = np.argsort(similarity)[::-1][:4]
    output = list()
    for name in np.array(corpus_tag)[topn_indices]:
        output.append("".join(name.split()))
    tfid_output_lst.append(output)
    return tfid_output_lst


def search(text,key):
    i = -1
    while True:
        if text[i] == key:
            return i
        i += -1


def matching(max_ratio_lst,output_lst,target_lst):

    for i,ratio in tqdm(enumerate(max_ratio_lst)):
        if ratio < 1.0:
            output_text = replace_(output_lst[i])
            output_text = tagger.parse(output_text).split()
            target_text = replace_(target_lst[i])
            target_text = tagger.parse(target_text).split()
            if "病院" in output_text and "病院" in target_text:
                if (output_text.count("病院")==2) and (target_text.count("病院")==2):
                    num1 = search(output_text,"病院")
                    num2 = search(target_text,"病院")
                    if output_text[num1 -1] == target_text[num2 -1] == '中央':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '総合':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '記念':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '市民':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '十字':
                        if output_text[num1 -3] != target_text[num2 -3]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '大学':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    else:
                        if output_text[num1 -1] != target_text[num2 -1]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                else:    
                    num1 = output_text.index("病院")
                    num2 = target_text.index("病院")
                    if output_text[num1 -1] == target_text[num2 -1] == '中央':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '総合':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '記念':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '市民':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '十字':
                        if output_text[num1 -3] != target_text[num2 -3]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    elif output_text[num1 -1] == target_text[num2 -1] == '大学':
                        if output_text[num1 -2] != target_text[num2 -2]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    else:
                        if output_text[num1 -1] != target_text[num2 -1]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
            elif "クリニック" in output_text and "クリニック" in target_text:
                num1 = output_text.index("クリニック")
                num2 = target_text.index("クリニック")
                if output_text[num1 -1] == target_text[num2 -1] == '内科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '外科':
                    if output_text[num1 -2] == target_text[num2 -2] == '整形':
                        if output_text[num1 -3] != target_text[num2 -3]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '眼科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '歯科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                else:
                    if output_text[num1 -1] != target_text[num2 -1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
            
            elif "医院" in output_text and "医院" in target_text:
                num1 = output_text.index("医院")
                num2 = target_text.index("医院")
                if output_text[num1 -1] == target_text[num2 -1] == '内科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '外科':
                    if output_text[num1 -2] == target_text[num2 -2] == '整形':
                        if output_text[num1 -3] != target_text[num2 -3]:
                            output_lst[i:i+1] = "_"
                        else:
                            continue
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '眼科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif output_text[num1 -1] == target_text[num2 -1] == '歯科':
                    if output_text[num1 -2] != target_text[num2 -2]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                else:
                    if output_text[num1 -1] != target_text[num2 -1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
            elif "薬局" in output_text and "薬局" in target_text:
                if (output_text.count("薬局")==2) and (target_text.count("薬局")==2):
                    num1 = search(output_text,"薬局")
                    num2 = search(target_text,"薬局")
                    if output_text[num1 -1] == target_text[num2 -1] == '調剤':
                        if (output_text[num1 -2] != output_text[-1]) and (target_text[num2 -2] != target_text[-1]):
                            if output_text[num1 -2] != target_text[num2 -2]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                        elif (output_text[num1 -2] != output_text[-1]) or (target_text[num2 -2] != target_text[-1]):
                            max_ratio_lst[i:i+1] = "_"
                        else:
                            if output_text[num1 +1] != target_text[num2 +1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                    else:
                        if (output_text[num1 -1] != output_text[-1]) and (target_text[num2 -1] != target_text[-1]):
                            if output_text[num1 -1] != target_text[num2 -1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                        elif (output_text[num1 -1] != output_text[-1]) or (target_text[num2 -1] != target_text[-1]):
                            max_ratio_lst[i:i+1] = "_"
                        else:
                            if output_text[num1 +1] != target_text[num2 +1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                else:
                    num1 = output_text.index("薬局")
                    num2 = target_text.index("薬局")
                    if output_text[num1 -1] == target_text[num2 -1] == '調剤':
                        if (output_text[num1 -2] != output_text[-1]) and (target_text[num2 -2] != target_text[-1]):
                            if output_text[num1 -2] != target_text[num2 -2]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                        elif (output_text[num1 -2] != output_text[-1]) or (target_text[num2 -2] != target_text[-1]):
                            max_ratio_lst[i:i+1] = "_"
                        else:
                            if output_text[num1 +1] != target_text[num2 +1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                    else:
                        if (output_text[num1 -1] != output_text[-1]) and (target_text[num2 -1] != target_text[-1]):
                            if output_text[num1 -1] != target_text[num2 -1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
                        elif (output_text[num1 -1] != output_text[-1]) or (target_text[num2 -1] != target_text[-1]):
                            max_ratio_lst[i:i+1] = "_"
                        else:
                            if output_text[num1 +1] != target_text[num2 +1]:
                                output_lst[i:i+1] = "_"
                            else:
                                continue
            elif "診療所" in output_text and "診療所" in target_text:
                num1 = output_text.index("診療所")
                num2 = target_text.index("診療所")
                if output_text[num1 -1] != target_text[num2 -1]:
                    output_lst[i:i+1] = "_"
                else:
                    continue
            elif "ファーマシー" in output_text and "ファーマシー" in target_text:
                num1 = output_text.index("ファーマシー")
                num2 = target_text.index("ファーマシー")
                if (output_text[num1 -1] != output_text[-1]) and (target_text[num2 -1] != target_text[-1]):
                    if output_text[num1 -1] != target_text[num2 -1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif (output_text[num1 -1] != output_text[-1]) or (target_text[num2 -1] != target_text[-1]):
                    max_ratio_lst[i:i+1] = "_"
                else:
                    if output_text[num1 +1] != target_text[num2 +1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
            elif "ドラッグ" in output_text and "ドラッグ" in target_text:
                num1 = output_text.index("ドラッグ")
                num2 = target_text.index("ドラッグ")
                if (output_text[num1 -1] != output_text[-1]) and (target_text[num2 -1] != target_text[-1]):
                    if output_text[num1 -1] != target_text[num2 -1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
                elif (output_text[num1 -1] != output_text[-1]) or (target_text[num2 -1] != target_text[-1]):
                    max_ratio_lst[i:i+1] = "_"
                else:
                    if output_text[num1 +1] != target_text[num2 +1]:
                        output_lst[i:i+1] = "_"
                    else:
                        continue
            elif "店" in output_text and "店" in target_text:
                num1 = output_text.index("店")
                num2 = target_text.index("店")
                if output_text[num1 -1] != target_text[num2 -1]:
                    output_lst[i:i+1] = "_"
                else:
                    continue
            elif ("病院" in output_text and "病院" not in target_text) or("病院" in target_text and "病院" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("医院" in output_text and "医院" not in target_text) or("医院" in target_text and "医院" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("クリニック" in output_text and "クリニック" not in target_text) or("クリニック" in target_text and "クリニック" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("診療所" in output_text and "診療所" not in target_text) or("診療所" in target_text and "診療所" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("薬局" in output_text and "薬局" not in target_text) or("薬局" in target_text and "薬局" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("ファーマシー" in output_text and "ファーマシー" not in target_text) or("ファーマシー" in target_text and "ファーマシー" not in output_text) :
                output_lst[i:i+1] = "_"
            elif ("ドラッグ" in output_text and "ドラッグ" not in target_text) or("ドラッグ" in target_text and "ドラッグ" not in output_text) :
                output_lst[i:i+1] = "_"
            else:
                continue
    return output_lst


In [3]:
# main

target_df = pd.read_excel("target.xlsx")
target_lst =  target_df["検索ワード"].values
corpus = codecs.open('corpus.txt', 'r', 'utf-8').read().splitlines()
corpus_tag = [tagger.parse(sentence).strip() for sentence in corpus]

max_ratio_lst = list()
output_lst = list()

# layer1
tfid_output_lst = tf_idf(corpus_tag, target_lst)

# layer2
for i,target in tqdm(enumerate(target_lst)):
    target = replace_(target)
    target = tagger.parse(target).strip()
    ratio_lst = list()
    for base in tfid_output_lst[i]:
        replace_(base)
        base = tagger.parse(base).strip()
        ratio = similar(target, base)
        ratio_lst.append(ratio)

    max_value = max(ratio_lst)
    max_ratio_lst.append(max_value)
    max_index = ratio_lst.index(max_value)
    output_lst.append(tfid_output_lst[i][max_index])

# layer3
output_lst = matching(max_ratio_lst,output_lst,target_lst)

# matching adress
adress_ratio = list()
for target,output in tqdm(zip(target_lst,output_lst)):
    if output == "_" or type(output) == float:
        adress_ratio.append("_")
    else:
        target = target[-10:]
        output = output[-10:]
        target = tagger.parse(target).strip()
        output = tagger.parse(output).strip()
        ratio = similar(target, output)
        adress_ratio.append(ratio)
        
output_df = pd.DataFrame({
    "results": output_lst,
    "ratio1": max_ratio_lst,
    "ratio2": adress_ratio
})
output_df.to_excel('output.xlsx', sheet_name='new_sheet_name',encoding="utf-8")